# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

In [1]:
!pip install geocoder

In [2]:
!pip install geopy

In [3]:
# Request Stuttgart's coordinate
import geocoder
from geopy.geocoders import Nominatim

address = 'Stuttgart, Germany'
geolocator = Nominatim(user_agent="st_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stuttgart are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Stuttgart are 48.7784485, 9.1800132.


In [4]:
# Assign the coordinate to Stuttgart center
stuttgart_center = (latitude, longitude)
# Define the bounding box of Stuttgart by the coordinate in the two conners of the box 
bounding_box = [(48.692019, 9.038601), (48.866399, 9.31582)]

In [5]:
# Create a grid of area candidates inside the bounding box, equaly spaced and visualize them
import folium
map_stutgart = folium.Map(location=stuttgart_center, zoom_start=13)
folium.Marker(stuttgart_center, popup='stuttgart_center').add_to(map_stutgart)
folium.Rectangle(bounds = bounding_box, radius=300, color='blue', fill=True, fill_color = 'red').add_to(map_stutgart)
no_of_circle_horizontal = 30
no_of_circle_vertical = 30
horizontal_sep = (bounding_box[1][0] - bounding_box[0][0])/(no_of_circle_horizontal-2)
vertical_sep = (bounding_box[1][1] - bounding_box[0][1])/(no_of_circle_vertical-2)
bound_lats = [bounding_box[0][0] + x*horizontal_sep for x in range(no_of_circle_horizontal-1)]
bound_lons = [bounding_box[0][1] + x*vertical_sep for x in range(no_of_circle_vertical-1)]
circle_center_lat_lon = []
for lat in bound_lats:
    for lon in bound_lons:
        circle_center_lat_lon.append((lat,lon))
for coordinate in circle_center_lat_lon:
    folium.Circle(location=coordinate, radius = 500).add_to(map_stutgart)
map_stutgart

In [6]:
# Create a function to request venues in the neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    CLIENT_ID = '2BHHDRJ3ZDKLQ4AWD5HEZPR1ENWU1SUALFHI3FONNA1XKDGF'
    CLIENT_SECRET = 'DBUBPZZYF1E4XRNGEBB3GCRIRGQ22P53NWU0KDDX4GR15ORX'
    VERSION = '20200324'
    LIMIT = '50'
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
# Run the function to get the venues for each neighborhood generated
import requests
import pandas as pd
names = [i for i in range(len(circle_center_lat_lon))]
latitudes, longitudes = zip(*circle_center_lat_lon)
all_venues = getNearbyVenues(names,latitudes, longitudes)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [85]:
len(all_venues)

3658

In [84]:
# Remove duplicate in the intersection of neighborhoods if available 
all_venues = all_venues.drop_duplicates()
len(all_venues)

3658

In [17]:
all_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,48.692019,9.038601,Spatzensee,48.694011,9.043653,Playground
1,0,48.692019,9.038601,café im krankenhaus,48.690379,9.032925,Café
2,1,48.692019,9.048502,Initiativpark,48.692467,9.052749,Athletics & Sports
3,1,48.692019,9.048502,Stadtwald Böblingen,48.688682,9.052283,Trail
4,7,48.692019,9.107906,Waldrefugien,48.693604,9.108142,Forest


In [11]:
# Write to .csv file to avoid requesting again when needed
all_venues.to_csv('stuttgart.csv')

In [12]:
# Examine the venue category to define favourite and non-favourite places
all_venues['Venue Category'].unique()

array(['Playground', 'Café', 'Athletics & Sports', 'Trail', 'Forest',
       'Supermarket', 'Bakery', 'Basketball Court', 'Bank', 'Pizza Place',
       'Gas Station', 'Shopping Mall', 'Plaza', 'Gym / Fitness Center',
       'Italian Restaurant', 'Bed & Breakfast', 'Hotel',
       'Spanish Restaurant', 'Brewery', 'Thai Restaurant',
       'Indian Restaurant', 'German Restaurant', 'Light Rail Station',
       'Motorcycle Shop', 'Train Station', 'Rental Car Location',
       'Swiss Restaurant', 'Park', 'Asian Restaurant', 'Currywurst Joint',
       'Coffee Shop', 'Pharmacy', 'Restaurant', 'Airport Service',
       'Fast Food Restaurant', 'Airport', 'Airport Lounge', 'Beer Garden',
       'Bus Station', 'River', 'Airport Terminal', 'BBQ Joint',
       'Scenic Lookout', 'Business Service', 'Food Truck', 'Pub',
       'Truck Stop', 'Seafood Restaurant', 'Stables', 'Church',
       'Kebab Restaurant', 'Bus Stop', 'Food Service',
       'French Restaurant', 'Taverna', 'Gym Pool', 'Diner',
    

In [75]:
# Create a dataframe of favourite categories
fav_cat = ['Gas Station','Thai Restaurant','Indian Restaurant','Train Station','Asian Restaurant' \
            'Restaurant','Bus Station','Seafood Restaurant','Bus Stop','Sushi Restaurant','Chinese Restaurant','Japanese Restaurant'\
            'Metro Station','Vietnamese Restaurant','Japanese Curry Restaurant','South Indian Restaurant','Korean Restaurant']
favorite = all_venues.loc[all_venues['Venue Category'].isin(fav_cat)]
favorite

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
14,10,48.692019,9.137608,Shell,48.694024,9.138979,Gas Station
31,13,48.692019,9.167310,Thang Long,48.688673,9.167308,Thai Restaurant
32,13,48.692019,9.167310,Kashmir,48.695733,9.170968,Indian Restaurant
41,14,48.692019,9.177211,Stuttgart Flughafen Bahnhof,48.691604,9.182186,Train Station
72,16,48.692019,9.197012,Stuttgart Airport Busterminal (SAB),48.692009,9.197445,Bus Station
82,16,48.692019,9.197012,Fernbushaltestelle Stuttgart Flughafen / Messe,48.694136,9.192732,Bus Station
99,27,48.692019,9.305919,NORDSEE,48.693396,9.304835,Seafood Restaurant
106,28,48.692019,9.315820,H Denkendorf Weingartstraße,48.695231,9.314899,Bus Stop
113,39,48.698247,9.137608,Shell,48.694024,9.138979,Gas Station
138,42,48.698247,9.167310,Kashmir,48.695733,9.170968,Indian Restaurant


In [193]:
# create map of New York using latitude and longitude values
map_stuttgart = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng in zip(favorite['Venue Latitude'], favorite['Venue Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stuttgart)  
    
map_stuttgart

In [188]:
# Create dataframe of competitors
com_cat = ['Supermarket','Food & Drink Shop','Market','Shopping Mall']
competitor = all_venues.loc[all_venues['Venue Category'].isin(com_cat)]
competitor.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,8,48.692019,9.117806,EDEKA Jäger,48.692259,9.122363,Supermarket
10,9,48.692019,9.127707,EDEKA Jäger,48.692259,9.122363,Supermarket
15,10,48.692019,9.137608,Euro Park,48.695377,9.139741,Shopping Mall
18,11,48.692019,9.147508,EDEKA Bauer,48.694187,9.143790,Supermarket
30,13,48.692019,9.167310,ALDI SÜD,48.694981,9.168974,Supermarket


In [195]:
# Remove not relevant venues in competitors list
import pandas
import numpy
pandas.options.mode.chained_assignment = None
not_relevant = ['toom Baumarkt','Bäckerei & Konditorei Bausch','Benz Wein- und Getränkemarkt','Ümmet',\
                'Gärtnerei & Hofladen Hönes','Landmetzgerei Brosi']

# Get names of indexes for which column venue has value in the not_relevant list
indexNames = competitor.loc[competitor['Venue'].isin(not_relevant)].index
# Delete these row indexes from dataFrame
competitor.drop(indexNames,inplace=True)
competitor.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,8,48.692019,9.117806,EDEKA Jäger,48.692259,9.122363,Supermarket
10,9,48.692019,9.127707,EDEKA Jäger,48.692259,9.122363,Supermarket
15,10,48.692019,9.137608,Euro Park,48.695377,9.139741,Shopping Mall
18,11,48.692019,9.147508,EDEKA Bauer,48.694187,9.143790,Supermarket
30,13,48.692019,9.167310,ALDI SÜD,48.694981,9.168974,Supermarket


In [194]:
# create map of New York using latitude and longitude values
map_stuttgart = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng in zip(competitor['Venue Latitude'], competitor['Venue Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stuttgart)  
    
map_stuttgart

In [47]:
# Haversine formular to calculate the distance to favorite and non-favorite locations
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [48]:
haversine(9.038601,  48.692019, 9.31582, 48.866399)

28.08174426255975

In [50]:
# Define candidates' coordinates
candidate_latitudes, candidate_longitudes = zip(*circle_center_lat_lon)

In [108]:
# Calculate the minimum distance to competitors
competitor_latitudes = competitor['Venue Latitude'].tolist()
competitor_longitudes = competitor['Venue Longitude'].tolist()

min_dist_to_competitor = []
for i in range(len(candidate_latitudes)):
    min_distance = 10000
    for j in range(len(competitor_latitudes)):
        distance = haversine(candidate_longitudes[i],  candidate_latitudes[i], competitor_longitudes[j], competitor_latitudes[j])
        if distance < min_distance:
            min_distance = distance
    min_dist_to_competitor.append(min_distance)

In [109]:
candidate_to_competitor_df.head()

,candidate_index,candidate_latitudes,candidate_longitudes,min_dist_to_competitor
0,0,48.692019,9.038601,28.054333
1,1,48.692019,9.048502,27.531838
2,2,48.692019,9.058402,27.018719
3,3,48.692019,9.068303,26.515521
4,4,48.692019,9.078204,26.022819


In [118]:
# Calculate the minimum distance to favorite places
fav_latitudes = favorite['Venue Latitude'].tolist()
fav_longitudes = favorite['Venue Longitude'].tolist()

min_dist_to_favourites = []
for i in range(len(candidate_latitudes)):
    min_distance = 10000
    for j in range(len(fav_latitudes)):
        distance = haversine(candidate_longitudes[i],  candidate_latitudes[i], fav_longitudes[j], fav_latitudes[j])
        if distance < min_distance:
            min_distance = distance
    min_dist_to_favourites.append(min_distance)

In [125]:
candidate = pd.DataFrame(list(zip(candidate_index, candidate_latitudes, candidate_longitudes, min_dist_to_competitor, 
                                  min_dist_to_favourites)), columns =['candidate_index', 'candidate_latitudes',
                                  'candidate_longitudes', 'min_dist_to_competitor','min_dist_to_favourites']) 

In [126]:
candidate.head()

,candidate_index,candidate_latitudes,candidate_longitudes,min_dist_to_competitor,min_dist_to_favourites
0,0,48.692019,9.038601,1.438573,1.889167
1,1,48.692019,9.048502,1.427683,1.677089
2,2,48.692019,9.058402,1.750152,1.764147
3,3,48.692019,9.068303,2.267984,2.113697
4,4,48.692019,9.078204,2.877574,2.622809


In [129]:
# Fillter the potential candiates which have locations with more than 2km to competitors and less than 0.5km to favourite places
final_candidates_df = candidate.loc[(candidate['min_dist_to_competitor'] > 2)  & (candidate['min_dist_to_favourites'] < 0.5)]

In [131]:
final_candidates_df

,candidate_index,candidate_latitudes,candidate_longitudes,min_dist_to_competitor,min_dist_to_favourites
121,121,48.716930,9.088104,2.207327,0.395566
411,411,48.779209,9.088104,2.719427,0.256864
436,436,48.785437,9.048502,2.174846,0.409731


In [133]:
!pip install reverse_geocoder

  Created wheel for reverse-geocoder: filename=reverse_geocoder-1.5.1-py3-none-any.whl size=2268099 sha256=35ce8f5b222f3ba8922c5323b72b9c091dfe3f314c5d7fb2132735836d3a3176
  Stored in directory: c:\users\phant\appdata\local\pip\cache\wheels\48\1e\42\7cda53ab4a7e6de12338278897c6d4dbab7c037989efdb277c
Successfully built reverse-geocoder


In [167]:
# Get the address of the potential candidate
import reverse_geocoder as rg 
  
def reverseGeocode(coordinates): 
    result = rg.search(coordinates) 
      
    # result is a list containing ordered dictionary. 
    return result
    
coordinates = list(zip(final_candidates_df['candidate_latitudes'].tolist(), final_candidates_df['candidate_longitudes'].tolist()))
addresses = reverseGeocode(coordinates)

In [168]:
addresses

[OrderedDict([('lat', '48.66667'),
              ('lon', '9.11667'),
              ('name', 'Steinenbronn'),
              ('admin1', 'Baden-Wuerttemberg'),
              ('admin2', 'Regierungsbezirk Stuttgart'),
              ('cc', 'DE')]),
 OrderedDict([('lat', '48.79954'),
              ('lon', '9.06316'),
              ('name', 'Gerlingen'),
              ('admin1', 'Baden-Wuerttemberg'),
              ('admin2', 'Regierungsbezirk Stuttgart'),
              ('cc', 'DE')]),
 OrderedDict([('lat', '48.79954'),
              ('lon', '9.06316'),
              ('name', 'Gerlingen'),
              ('admin1', 'Baden-Wuerttemberg'),
              ('admin2', 'Regierungsbezirk Stuttgart'),
              ('cc', 'DE')])]

In [181]:
can1 = (addresses[0]['name']+ ',' +addresses[0]['admin2'])
can2 = (addresses[1]['name']+ ',' +addresses[1]['admin2'])
can3 = (addresses[2]['name']+ ',' +addresses[2]['admin2'])

In [183]:
Addresses = [can1,can2,can3]

In [184]:
final_candidates_df['Address'] = Addresses

In [185]:
final_candidates_df

,candidate_index,candidate_latitudes,candidate_longitudes,min_dist_to_competitor,min_dist_to_favourites,Address
121,121,48.716930,9.088104,2.207327,0.395566,"Steinenbronn,Regierungsbezirk Stuttgart"
411,411,48.779209,9.088104,2.719427,0.256864,"Gerlingen,Regierungsbezirk Stuttgart"
436,436,48.785437,9.048502,2.174846,0.409731,"Gerlingen,Regierungsbezirk Stuttgart"


In [192]:
# create map of New York using latitude and longitude values
map_stuttgart = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng in zip(final_candidates_df['candidate_latitudes'], final_candidates_df['candidate_longitudes']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stuttgart)  
    
map_stuttgart